# ⏱️📈 Example 6: Chebyshev polynomials performance comparison

This notebook compares the performance of the Chebyshev polynomials implemented in 
Numpy and Numba.

## 1) ☁️🔌 Imports


In [ ]:
import os

import numpy as np
import perfplot
from matplotlib import pyplot as plt

from robust_fourier import (
    chebyshev_poly_basis,
)

%matplotlib widget

## 2) ⚙️📝 Configuration


In [ ]:
# the x-values to evaluate the Chebyshev polynomials
X_FROM = -1.0
X_TO = 1.0
NUM_X = 2_501

# the scaling factor alpha to use
ALPHA = 1.0

# the orders of the Chebyshev polynomials to evaluate
ORDERS = [int(2 ** (i * 0.3333)) for i in range(0, 45)]

# the path where to save the plot
SAVE_PATH = "../docs/chebyshev_polynomials/EX-06-ChebyshevPolynomialsPerformance.svg"
# the resolution of the saved plot
DPI = 300

# the Numba-based version of the Chebyshev polynomials is pre-compiled to avoid the
# compilation overhead in the performance comparison
_ = chebyshev_poly_basis(
    x=np.linspace(start=X_FROM, stop=X_TO, num=NUM_X),
    n=ORDERS[0],
    alpha=ALPHA,
    kind=1,
    jit=True,
)

## 3) 🏃🦾 Performance comparison


In [ ]:
plt.close("all")

perfplot_kwargs = dict(
    setup=lambda n: (np.linspace(start=X_FROM, stop=X_TO, num=NUM_X), n, ALPHA),
    kernels=[
        lambda x, n, alpha: chebyshev_poly_basis(
            x=x,
            n=n,
            alpha=alpha,
            kind=1,
            jit=False,
        ),
        lambda x, n, alpha: chebyshev_poly_basis(
            x=x,
            n=n,
            alpha=alpha,
            kind=1,
            jit=True,
        ),
    ],
    labels=[
        "NumPy Full Basis",
        "Numba Full Basis",
    ],
    n_range=ORDERS,
    logx=True,
    logy=True,
    xlabel="Order of the Chebyshev Polynomial Basis",
    title=f"Performance of Chebyshev Polynomials for {NUM_X} x-values",
    show_progress=True,
    target_time_per_measurement=0.1,
)


perfplot.show(**perfplot_kwargs)  # type: ignore

fig = plt.gcf()
ax = plt.gca()
ax.grid(which="both", axis="both")
# make the spines visible black and wider
for spine in ax.spines.values():
    spine.set_visible(True)
    spine.set_edgecolor("black")
    spine.set_linewidth(0.5)

ax.xaxis.label.set_color("black")
ax.yaxis.label.set_color("black")
ax.tick_params(axis="x", colors="black")
ax.tick_params(axis="y", colors="black")

ax.title.set_color("black")

fig.set_size_inches(10, 6)
fig.tight_layout()

if os.getenv("ROBFT_DEVELOPER", "false").lower() == "true":
    plt.savefig(
        bbox_inches="tight",
        fname=os.path.join(os.getcwd(), SAVE_PATH),
        dpi=DPI,
    )